In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from concurrent.futures import ThreadPoolExecutor
from threading import Lock


urls = [
    "https://null-byte.wonderhowto.com/how-to/enumerate-smb-with-enum4linux-smbclient-0198049/",
    "https://null-byte.wonderhowto.com/how-to/exploit-eternalblue-windows-server-with-metasploit-0195413/",
    "https://null-byte.wonderhowto.com/how-to/automate-wi-fi-hacking-with-wifite2-0191739/",
    ]

lock = Lock()

def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    data = []


    sections = soup.find_all('h2', class_='step')
    for section in sections:
        section_title = section.text.strip()

        prompt = None

        steps = section.find_all_next(['p', 'pre'])
        for index, step in enumerate(steps, start=1):
            if step.name == 'p':
                prompt = step.text.strip()
            elif step.name == 'pre' and 'syntax-highlighted' in step.get('class', []):
                response = step.text.strip()

                data.append([url, f"{section_title} - Step {index}", prompt, response])

    return data

with ThreadPoolExecutor(max_workers=5) as executor:
    all_data = []
    for data in executor.map(scrape_page, urls):
        all_data.extend(data)

csv_filename = 'dataset_combined.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)

    csv_writer.writerow(["Website Link", "Step Number", "Prompt", "Response"])

    with lock:
        csv_writer.writerows(all_data)
